<a href="https://colab.research.google.com/github/DanielRaiicHu/TelecomX/blob/main/TelecomX.ipynb?forceRefresh=true">
  <img src="https://drive.google.com/uc?export=view&id=191Woxwut0iaIL2hCcCIeXRSB6oba_88H" alt="Abrir en Colab" width="120"/>
</a>

## **Archivo del proyecto**

https://raw.githubusercontent.com/alura-cursos/challenge2-data-science-LATAM/refs/heads/main/TelecomX_Data.json

Exploración básica
```
df.shape           # Dimensiones del DataFrame
df.columns         # Lista de nombres de columnas
df.head()          # Primeras 5 filas
df.info()          # Resumen de tipos de datos y valores nulos
df.describe()      # Estadísticas numéricas (si hay números)
df.dtypes          # Tipo de dato por columna
df.isnull().sum()  # Conteo de valores nulos por columna
```



## **Extracción de datos**

In [223]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


### **Leyendo los datos**

In [224]:
url = 'https://raw.githubusercontent.com/alura-cursos/challenge2-data-science-LATAM/refs/heads/main/TelecomX_Data.json'

In [225]:
df = pd.read_json(url)

In [226]:
df.head()

,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


### **Normalizando el dataframe**

Debemos normalizar el dataframe ya que presenta información anidada.
Sobreescribiremos el dataframe.

In [227]:
# Normalizo cada una de las columnas que contienen diccionarios
df_customer = pd.json_normalize(df['customer'])
df_phone = pd.json_normalize(df['phone'])
df_internet = pd.json_normalize(df['internet'])
df_account = pd.json_normalize(df['account'])

# Concateno las columnas con las columnas normalizadas en el dataframe
df = pd.concat([df[['customerID', 'Churn']], df_customer, df_phone, df_internet, df_account], axis=1)
df.head()

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4


## **Análisis de los datos**

In [228]:
# Dimensiones del Datafrane
df.shape

(7267, 21)

In [229]:
# Las columnas que tenemos en el dataframe
df.columns

Index(['customerID', 'Churn', 'gender', 'SeniorCitizen', 'Partner',
       'Dependents', 'tenure', 'PhoneService', 'MultipleLines',
       'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
       'PaperlessBilling', 'PaymentMethod', 'Charges.Monthly',
       'Charges.Total'],
      dtype='object')

In [230]:
# Para ver la información relevante y tipos de datos de las columnas
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7267 non-null   object 
 1   Churn             7267 non-null   object 
 2   gender            7267 non-null   object 
 3   SeniorCitizen     7267 non-null   int64  
 4   Partner           7267 non-null   object 
 5   Dependents        7267 non-null   object 
 6   tenure            7267 non-null   int64  
 7   PhoneService      7267 non-null   object 
 8   MultipleLines     7267 non-null   object 
 9   InternetService   7267 non-null   object 
 10  OnlineSecurity    7267 non-null   object 
 11  OnlineBackup      7267 non-null   object 
 12  DeviceProtection  7267 non-null   object 
 13  TechSupport       7267 non-null   object 
 14  StreamingTV       7267 non-null   object 
 15  StreamingMovies   7267 non-null   object 
 16  Contract          7267 non-null   object 


In [231]:
# Revisión de datos únicos para cada columna
for col in df.columns:
    if df[col].dtype == 'object':
        print(f"Valores únicos en '{col}':")
        print(df[col].unique())
        print("—" * 40)

Valores únicos en 'customerID':
['0002-ORFBO' '0003-MKNFE' '0004-TLHLJ' ... '9992-UJOEL' '9993-LHIEB'
 '9995-HOTOH']
————————————————————————————————————————
Valores únicos en 'Churn':
['No' 'Yes' '']
————————————————————————————————————————
Valores únicos en 'gender':
['Female' 'Male']
————————————————————————————————————————
Valores únicos en 'Partner':
['Yes' 'No']
————————————————————————————————————————
Valores únicos en 'Dependents':
['Yes' 'No']
————————————————————————————————————————
Valores únicos en 'PhoneService':
['Yes' 'No']
————————————————————————————————————————
Valores únicos en 'MultipleLines':
['No' 'Yes' 'No phone service']
————————————————————————————————————————
Valores únicos en 'InternetService':
['DSL' 'Fiber optic' 'No']
————————————————————————————————————————
Valores únicos en 'OnlineSecurity':
['No' 'Yes' 'No internet service']
————————————————————————————————————————
Valores únicos en 'OnlineBackup':
['Yes' 'No' 'No internet service']
————————————————————

In [232]:
# Analizando datos
df['Charges.Total'][0]

'593.3'

In [233]:
# Tipos de datos
type(df['SeniorCitizen'][0])

numpy.int64

In [234]:
# Ver las filas con el campo en blanco en la columna Churn
df[df['Churn'] == '']

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
30,0047-ZHDTW,,Female,0,No,No,11,Yes,Yes,Fiber optic,...,No,No,No,No,No,Month-to-month,Yes,Bank transfer (automatic),79.00,929.3
75,0120-YZLQA,,Male,0,No,No,71,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,Yes,Credit card (automatic),19.90,1355.1
96,0154-QYHJU,,Male,0,No,No,29,Yes,No,DSL,...,Yes,No,Yes,No,No,One year,Yes,Electronic check,58.75,1696.2
98,0162-RZGMZ,,Female,1,No,No,5,Yes,No,DSL,...,Yes,No,Yes,No,No,Month-to-month,No,Credit card (automatic),59.90,287.85
175,0274-VVQOQ,,Male,1,Yes,No,65,Yes,Yes,Fiber optic,...,Yes,Yes,No,Yes,Yes,One year,Yes,Bank transfer (automatic),103.15,6792.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7158,9840-GSRFX,,Female,0,No,No,14,Yes,Yes,DSL,...,Yes,No,No,No,No,One year,Yes,Mailed check,54.25,773.2
7180,9872-RZQQB,,Female,0,Yes,No,49,No,No phone service,DSL,...,No,No,No,Yes,No,Month-to-month,No,Bank transfer (automatic),40.65,2070.75
7211,9920-GNDMB,,Male,0,No,No,9,Yes,Yes,Fiber optic,...,No,No,No,No,No,Month-to-month,Yes,Electronic check,76.25,684.85
7239,9955-RVWSC,,Female,0,Yes,Yes,67,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,Yes,Bank transfer (automatic),19.25,1372.9


In [235]:
# Cantidad de campos en blanco en la columna de Churn
(df['Churn'] == '').sum()

np.int64(224)

### **Análisis y decisiones sobre la limpieza de datos**

Tras revisar individualmente cada columna del conjunto de datos, se llegaron a las siguientes conclusiones:

* Columnas binarias (`'Partner'`, `'Dependents'`, `'PhoneService'` y `'PaperlessBilling'`) utilizan los valores `'Yes'` y `'No'`. Aunque podrían transformarse a booleanos (`True/False`) para facilitar operaciones lógicas o modelado, en este análisis se decidió mantener su formato original para favorecer la legibilidad.

* La columna `'SeniorCitizen'`, a diferencia de las anteriores, utiliza valores `0` y `1` de tipo `int64`. Para mantener un formato coherente con las demás variables binarias, se optó por **reemplazar 0 por** `'No'` **y 1 por**` 'Yes'`, y convertir la columna a tipo **string**.

* En la columna `'Churn'`, que indica si un cliente dejó o no la empresa, se detectaron tres valores únicos: `'Yes'`, `'No'` y `''` (cadena vacía). Dado que el valor vacío representa un estado indefinido, se decidió **eliminar estas filas para evitar inconsistencias analíticas**. El total de filas a eliminar es de **224**.

* Finalmente, la columna `'Charges.Total'`, que originalmente estaba en formato `object`, será convertida a `float64`, ya que contiene datos numéricos necesarios para cálculos posteriores.

---
---
**Ya que los datos están en inglés, se decidió hacer una modificación a los nombres de las columnas para mayor legibilidad**

| Nombre original       | Nuevo nombre       |
|------------------------|--------------------------|
| customerID             | ID_Cliente               |
| Churn                  | Evasion                 |
| gender                 | Genero                   |
| SeniorCitizen          | AdultoMayor              |
| Partner                | TienePareja              |
| Dependents             | Dependientes             |
| tenure                 | MesesAntiguedad          |
| PhoneService           | ServicioTelefonico       |
| MultipleLines          | LineasMultiples          |
| InternetService        | ServicioInternet         |
| OnlineSecurity         | SeguridadEnLinea         |
| OnlineBackup           | RespaldoEnLinea          |
| DeviceProtection       | ProteccionDispositivo    |
| TechSupport            | SoporteTecnico           |
| StreamingTV            | StreamingTV              |
| StreamingMovies        | StreamingPeliculas       |
| Contract               | TipoContrato             |
| PaperlessBilling       | FacturaElectronica       |
| PaymentMethod          | MetodoPago               |
| Charges.Monthly        | CargoMensual             |
| Charges.Total          | CargoTotal               |

---
---
**Datos Internos que también deben ser modificados para mayor legibilidad**

| Valor original              | Nuevo valor                   |
|-----------------------------|------------------------------------------|
| Yes                         | Sí                                       |
| No                          | No                                       |
| Female                      | Femenino                                 |
| Male                        | Masculino                                |
| No phone service            | Sin servicio telefónico                  |
| No internet service         | Sin servicio de internet                 |
| DSL                         | DSL                                      |
| Fiber optic                 | Fibra óptica                             |
| Month-to-month              | Mensual                                  |
| One year                    | Un año                                   |
| Two year                    | Dos años                                 |
| Mailed check                | Cheque por correo                        |
| Electronic check            | Cheque electrónico                       |
| Credit card (automatic)     | Tarjeta de crédito (automático)          |
| Bank transfer (automatic)   | Transferencia bancaria (automática)      |

## **Transformación de datos**

### **Cambios en SeniorCitizen (AdultoMayor)**

In [236]:
# Reemplaza 0 y 1 por 'No' y 'Yes' para mayor legibilidad y mantener un formato uniforme.
df['SeniorCitizen'] = df['SeniorCitizen'].map({0: 'No', 1: 'Yes'})

### **Cambios en Charges.Total (Cargo Total)**

In [237]:
# nos damos cuenta que hay 11 valores vacíos en columna 'Charges.Total'
# Esto no nos deja pasar los valores de str a float64
(df['Charges.Total'] == ' ').sum()

np.int64(11)

In [238]:
df[df['Charges.Total'] == ' '].head()

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
975,1371-DWPAZ,No,Female,No,Yes,Yes,0,No,No phone service,DSL,...,Yes,Yes,Yes,Yes,No,Two year,No,Credit card (automatic),56.05,
1775,2520-SGTTA,No,Female,No,Yes,Yes,0,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.00,
1955,2775-SEFEE,No,Male,No,No,Yes,0,Yes,Yes,DSL,...,Yes,No,Yes,No,No,Two year,Yes,Bank transfer (automatic),61.90,
2075,2923-ARZLG,No,Male,No,Yes,Yes,0,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,One year,Yes,Mailed check,19.70,
2232,3115-CZMZD,No,Male,No,No,Yes,0,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.25,


In [239]:
# Los valores vacíos en 'Charges.Total' corresponden a clientes con tenure = 0.
# Por lo tanto, asignamos 0.0 como valor válido, ya que no se han generado cargos.
df['Charges.Total'] = df['Charges.Total'].replace(' ', 0.0).astype(float)

### **Cambios en Churn (Evasion)**

In [240]:
# Filtramos las filas válidas ('Sí' o 'No') y eliminamos las que tienen valor vacío ('')
df = df[df['Churn'].isin(['Yes', 'No'])].copy()
df.head()

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
0,0002-ORFBO,No,Female,No,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.30
1,0003-MKNFE,No,Male,No,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.40
2,0004-TLHLJ,Yes,Male,No,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,Yes,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,Yes,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.40


In [241]:
df[df['Churn'] == ''].head()

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total


In [242]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7043 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   Churn             7043 non-null   object 
 2   gender            7043 non-null   object 
 3   SeniorCitizen     7043 non-null   object 
 4   Partner           7043 non-null   object 
 5   Dependents        7043 non-null   object 
 6   tenure            7043 non-null   int64  
 7   PhoneService      7043 non-null   object 
 8   MultipleLines     7043 non-null   object 
 9   InternetService   7043 non-null   object 
 10  OnlineSecurity    7043 non-null   object 
 11  OnlineBackup      7043 non-null   object 
 12  DeviceProtection  7043 non-null   object 
 13  TechSupport       7043 non-null   object 
 14  StreamingTV       7043 non-null   object 
 15  StreamingMovies   7043 non-null   object 
 16  Contract          7043 non-null   object 
 17  

### **Transformación de nombres del inglés al español**

In [243]:
df.rename(columns={
    'customerID': 'ID_Cliente',
    'Churn': 'Evasion',
    'gender': 'Genero',
    'SeniorCitizen': 'AdultoMayor',
    'Partner': 'TienePareja',
    'Dependents': 'Dependientes',
    'tenure': 'MesesAntiguedad',
    'PhoneService': 'ServicioTelefonico',
    'MultipleLines': 'LineasMultiples',
    'InternetService': 'ServicioInternet',
    'OnlineSecurity': 'SeguridadEnLinea',
    'OnlineBackup': 'RespaldoEnLinea',
    'DeviceProtection': 'ProteccionDispositivo',
    'TechSupport': 'SoporteTecnico',
    'StreamingTV': 'StreamingTV',
    'StreamingMovies': 'StreamingPeliculas',
    'Contract': 'TipoContrato',
    'PaperlessBilling': 'FacturaElectronica',
    'PaymentMethod': 'MetodoPago',
    'Charges.Monthly': 'CargoMensual',
    'Charges.Total': 'CargoTotal'
}, inplace=True)

In [244]:
# Diccionario con las traducciones
valores_a_reemplazar = {
    'Yes': 'Sí',
    'No': 'No',
    '': 'Desconocido',
    'Female': 'Femenino',
    'Male': 'Masculino',
    'No phone service': 'Sin servicio telefónico',
    'No internet service': 'Sin servicio de internet',
    'DSL': 'DSL',
    'Fiber optic': 'Fibra óptica',
    'Month-to-month': 'Mensual',
    'One year': 'Un año',
    'Two year': 'Dos años',
    'Mailed check': 'Cheque por correo',
    'Electronic check': 'Cheque electrónico',
    'Credit card (automatic)': 'Tarjeta de crédito (automático)',
    'Bank transfer (automatic)': 'Transferencia bancaria (automática)'
}

# Aplicamos los reemplazos a todo el DataFrame
df.replace(valores_a_reemplazar, inplace=True)

### **Creamos la columna CuentasDiarias**

In [245]:
# Se obtiene la posición de 'CargoMensual'
pos = df.columns.get_loc('CargoMensual')

# Se agrega la columna 'CuentasDiarias', redondeada a 2 decimales, en la posición obtenida
df.insert(loc=pos, column='CuentasDiarias', value=(df['CargoMensual'] / 30).round(2))

### **Se reinicia el índice en el Dataframe**

In [246]:
# Se reinicia el índice del DataFrame y elimina el índice anterior para mantener un orden limpio y consecutivo
df.reset_index(inplace = True, drop = True)
df

,ID_Cliente,Evasion,Genero,AdultoMayor,TienePareja,Dependientes,MesesAntiguedad,ServicioTelefonico,LineasMultiples,ServicioInternet,...,ProteccionDispositivo,SoporteTecnico,StreamingTV,StreamingPeliculas,TipoContrato,FacturaElectronica,MetodoPago,CuentasDiarias,CargoMensual,CargoTotal
0,0002-ORFBO,No,Femenino,No,Sí,Sí,9,Sí,No,DSL,...,No,Sí,Sí,No,Un año,Sí,Cheque por correo,2.19,65.60,593.30
1,0003-MKNFE,No,Masculino,No,No,No,9,Sí,Sí,DSL,...,No,No,No,Sí,Mensual,No,Cheque por correo,2.00,59.90,542.40
2,0004-TLHLJ,Sí,Masculino,No,No,No,4,Sí,No,Fibra óptica,...,Sí,No,No,No,Mensual,Sí,Cheque electrónico,2.46,73.90,280.85
3,0011-IGKFF,Sí,Masculino,Sí,Sí,No,13,Sí,No,Fibra óptica,...,Sí,No,Sí,Sí,Mensual,Sí,Cheque electrónico,3.27,98.00,1237.85
4,0013-EXCHZ,Sí,Femenino,Sí,Sí,No,3,Sí,No,Fibra óptica,...,No,Sí,Sí,No,Mensual,Sí,Cheque por correo,2.80,83.90,267.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,9987-LUTYD,No,Femenino,No,No,No,13,Sí,No,DSL,...,No,Sí,No,No,Un año,No,Cheque por correo,1.84,55.15,742.90
7039,9992-RRAMN,Sí,Masculino,No,Sí,No,22,Sí,Sí,Fibra óptica,...,No,No,No,Sí,Mensual,Sí,Cheque electrónico,2.84,85.10,1873.70
7040,9992-UJOEL,No,Masculino,No,No,No,2,Sí,No,DSL,...,No,No,No,No,Mensual,Sí,Cheque por correo,1.68,50.30,92.75
7041,9993-LHIEB,No,Masculino,No,Sí,Sí,67,Sí,No,DSL,...,Sí,Sí,No,Sí,Dos años,No,Cheque por correo,2.26,67.85,4627.65


In [247]:
# Revisión de datos únicos para cada columna actualizazda
for col in df.columns:
    if df[col].dtype == 'object':
        print(f"Valores únicos en '{col}':")
        print(df[col].unique())
        print("—" * 40)

Valores únicos en 'ID_Cliente':
['0002-ORFBO' '0003-MKNFE' '0004-TLHLJ' ... '9992-UJOEL' '9993-LHIEB'
 '9995-HOTOH']
————————————————————————————————————————
Valores únicos en 'Evasion':
['No' 'Sí']
————————————————————————————————————————
Valores únicos en 'Genero':
['Femenino' 'Masculino']
————————————————————————————————————————
Valores únicos en 'AdultoMayor':
['No' 'Sí']
————————————————————————————————————————
Valores únicos en 'TienePareja':
['Sí' 'No']
————————————————————————————————————————
Valores únicos en 'Dependientes':
['Sí' 'No']
————————————————————————————————————————
Valores únicos en 'ServicioTelefonico':
['Sí' 'No']
————————————————————————————————————————
Valores únicos en 'LineasMultiples':
['No' 'Sí' 'Sin servicio telefónico']
————————————————————————————————————————
Valores únicos en 'ServicioInternet':
['DSL' 'Fibra óptica' 'No']
————————————————————————————————————————
Valores únicos en 'SeguridadEnLinea':
['No' 'Sí' 'Sin servicio de internet']
——————————————

###  **Cambios realizados**

* La columna `SeniorCitizen`, que contenía los valores 0 y 1, fue transformada para mejorar la legibilidad. Así, los valores fueron reemplazados por 'No' y 'Yes', respectivamente, y se cambió el tipo de dato a cadena de texto.

* La columna `Charges.Total` contenía algunos valores vacíos, los cuales se identificaron como correspondientes a clientes con **tenure = 0** (es decir, sin meses contratados aún). En estos casos, se concluyó que era lógico que el cargo total fuera 0, por lo que se reemplazaron los valores vacíos por 0.0 y se convirtió la columna a tipo float.

* Se realizó una revisión general de los tipos de datos, identificando que únicamente `tenure` y `Charges.Monthly` eran numéricos en origen. `Charges.Total` fue ajustado posteriormente para formar parte del análisis posterior.

* Finalmente, se optó por mantener los registros con `Charges.Total = 0`, ya que aunque estos clientes no han generado cargos aún, pueden ser relevantes en análisis relacionados con Evasion u otros comportamientos.

* Se detectaron registros en la columna `Churn` con valores vacíos (''). Estos casos no indicaban claramente si el cliente había abandonado o no la empresa, por lo que se decidió eliminar dichas filas para evitar ambigüedad en los análisis. El total de filas eliminadas fue de `224`.

* Se renombraron las columnas originales del dataframe al español para facilitar la lectura y el análisis. Por ejemplo, `customerID` pasó a ser `ID_Cliente`, `Churn` se renombró como `Evasion`, entre otras.
Desde ahora en adelante se mencionaran en el proyecto con sus nombres al español.

* Se agrega una nueva columna llamada `CuentasDiarias`

* Se hizo un reinicio en el índice, eliminando el índice anterior, para mantener el orden y que se vea visualmente más limpio.

## **Carga y análisis gráfico de los datos**

### **Guarda el dataframe**

In [248]:
# Guarda el dataframe en un archivo separado por comas (csv)
df.to_csv("telecom_x.csv", index=False)

### **Revisión de datos**

In [249]:
# Vuelvo a leer el dataframe para asegurarme que esté todo correcto
df = pd.read_csv("telecom_x.csv").head()
df

,ID_Cliente,Evasion,Genero,AdultoMayor,TienePareja,Dependientes,MesesAntiguedad,ServicioTelefonico,LineasMultiples,ServicioInternet,...,ProteccionDispositivo,SoporteTecnico,StreamingTV,StreamingPeliculas,TipoContrato,FacturaElectronica,MetodoPago,CuentasDiarias,CargoMensual,CargoTotal
0,0002-ORFBO,No,Femenino,No,Sí,Sí,9,Sí,No,DSL,...,No,Sí,Sí,No,Un año,Sí,Cheque por correo,2.19,65.6,593.30
1,0003-MKNFE,No,Masculino,No,No,No,9,Sí,Sí,DSL,...,No,No,No,Sí,Mensual,No,Cheque por correo,2.00,59.9,542.40
2,0004-TLHLJ,Sí,Masculino,No,No,No,4,Sí,No,Fibra óptica,...,Sí,No,No,No,Mensual,Sí,Cheque electrónico,2.46,73.9,280.85
3,0011-IGKFF,Sí,Masculino,Sí,Sí,No,13,Sí,No,Fibra óptica,...,Sí,No,Sí,Sí,Mensual,Sí,Cheque electrónico,3.27,98.0,1237.85
4,0013-EXCHZ,Sí,Femenino,Sí,Sí,No,3,Sí,No,Fibra óptica,...,No,Sí,Sí,No,Mensual,Sí,Cheque por correo,2.80,83.9,267.40


In [250]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ID_Cliente             5 non-null      object 
 1   Evasion                5 non-null      object 
 2   Genero                 5 non-null      object 
 3   AdultoMayor            5 non-null      object 
 4   TienePareja            5 non-null      object 
 5   Dependientes           5 non-null      object 
 6   MesesAntiguedad        5 non-null      int64  
 7   ServicioTelefonico     5 non-null      object 
 8   LineasMultiples        5 non-null      object 
 9   ServicioInternet       5 non-null      object 
 10  SeguridadEnLinea       5 non-null      object 
 11  RespaldoEnLinea        5 non-null      object 
 12  ProteccionDispositivo  5 non-null      object 
 13  SoporteTecnico         5 non-null      object 
 14  StreamingTV            5 non-null      object 
 15  StreamingP

### **Análisis gráfico de los datos**

In [251]:
# Analisis descriptivo
df.describe()

,MesesAntiguedad,CuentasDiarias,CargoMensual,CargoTotal
count,5.00000,5.000000,5.000000,5.000000
mean,7.60000,2.544000,76.260000,584.360000
std,4.09878,0.505203,15.148036,394.167384
min,3.00000,2.000000,59.900000,267.400000
25%,4.00000,2.190000,65.600000,280.850000
50%,9.00000,2.460000,73.900000,542.400000
75%,9.00000,2.800000,83.900000,593.300000
max,13.00000,3.270000,98.000000,1237.850000


## **Informe final**